In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM, Dense, Embedding
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
cv = CountVectorizer()
ps = PorterStemmer()
tfvec = TfidfVectorizer(max_features=5000)
import re
import nltk
nltk.download('stopwords')
import random

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('/content/drive/MyDrive/emails.csv')

In [3]:
df

,text,spam
0,Subject: - - - - - - - - - - - - - - - - - - -...,0
1,Subject: - - > direct marketing will increase ...,1
2,Subject: http : / / www . virtu ally - anyw...,1
3,"Subject: ! gorgeous , custom websites - $ 399 ...",1
4,"Subject: "" content - type : text / plain ; ch...",1
...,...,...
5721,"Subject: zimin ' s father vince and stinson ,...",0
5722,Subject: zingales seminar enron seminar serie...,0
5723,Subject: zingales seminar enron seminar serie...,0
5724,Subject: zingales seminar fyi ! - - - - - - ...,0


In [4]:
from sklearn.utils import shuffle
df = shuffle(df)
df = df.reset_index()

In [5]:
df.head(25)

,index,text,spam
0,1059,"Subject: eol dan , i am forwarding you the s...",0
1,5561,"Subject: when we say free , we mean free ! to...",1
2,528,Subject: candy super $ money maker you are re...,1
3,3810,Subject: re : mountaintop meetings next week ...,0
4,5617,Subject: year end 2000 performance feedback n...,0
5,265,Subject: alp presentation this will be in eb ...,0
6,5605,Subject: year end 2000 feedback deadline prc ...,0
7,5367,Subject: use this handy interest calculator to...,1
8,1951,"Subject: livelink access moyez , i could not...",0
9,5237,Subject: tiger evals - attachment tiger hosts...,0


In [6]:
corpus = []
for i in range(0,len(df['text'])):
    review = re.sub('[^a-zA-Z]',' ',df['text'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [7]:
df['cleaned'] = corpus

In [37]:
from sklearn.model_selection import train_test_split
xtr, xte, ytr, yte = train_test_split(df['cleaned'],df['spam'],test_size = 0.25,shuffle=True,random_state=42)

In [38]:
x = tfvec.fit_transform(xtr).toarray()
y = ytr

In [39]:
xte = tfvec.transform(xte).toarray()

In [11]:
from sklearn.naive_bayes import GaussianNB
cls = GaussianNB()
cls.fit(x,y)

GaussianNB(priors=None, var_smoothing=1e-09)

In [12]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x,y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [13]:
from sklearn.tree import DecisionTreeClassifier
dc_cls = DecisionTreeClassifier()
dc_cls.fit(x,y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [14]:
cls_pred = cls.predict(xte)
rf_pred = rf.predict(xte)
dc_pred = dc_cls.predict(xte)

In [15]:
from sklearn.metrics import accuracy_score
print(accuracy_score(cls_pred,yte))
print(accuracy_score(rf_pred,yte))
print(accuracy_score(dc_pred,yte))

0.9587988826815642
0.9839385474860335
0.9560055865921788


In [29]:
txt = ['Subject: re : summer associate mentor ']
for i in range(0,len(txt)):
  txt = re.sub('[^a-zA-Z]',' ',txt[i])
  txt = txt.lower()
  txt = txt.split()
  txt = [ps.stem(word) for word in txt if word not in stopwords.words('english')]
  txt = ' '.join(txt)

txt = [txt]


In [30]:
txt

['subject summer associ mentor']

In [31]:
txt = tfvec.transform(txt).toarray()

In [33]:
rf.predict(txt)

array([0])

In [25]:
import pickle

In [26]:
file = open('spam.pkl','wb')
pickle.dump(rf,file)

In [27]:
model = open('spam.pkl','rb')
spam = pickle.load(model)

In [40]:
spam.predict(xte)

array([0, 0, 1, ..., 0, 1, 0])